In [131]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from datetime import datetime
import time
import seaborn as sns

In [134]:
test = pd.read_csv('test_BdBKkAj.csv')
train = pd.read_csv('train_aWnotuB.csv')

In [135]:
test.info(), train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11808 entries, 0 to 11807
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   DateTime  11808 non-null  object
 1   Junction  11808 non-null  int64 
 2   ID        11808 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 276.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48120 entries, 0 to 48119
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   DateTime  48120 non-null  object
 1   Junction  48120 non-null  int64 
 2   Vehicles  48120 non-null  int64 
 3   ID        48120 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.5+ MB


(None, None)

In [136]:
print("TEST")
print(test.head())
print("TRAIN")
print(train.head())

TEST
              DateTime  Junction           ID
0  2017-07-01 00:00:00         1  20170701001
1  2017-07-01 01:00:00         1  20170701011
2  2017-07-01 02:00:00         1  20170701021
3  2017-07-01 03:00:00         1  20170701031
4  2017-07-01 04:00:00         1  20170701041
TRAIN
              DateTime  Junction  Vehicles           ID
0  2015-11-01 00:00:00         1        15  20151101001
1  2015-11-01 01:00:00         1        13  20151101011
2  2015-11-01 02:00:00         1        10  20151101021
3  2015-11-01 03:00:00         1         7  20151101031
4  2015-11-01 04:00:00         1         9  20151101041


In [137]:
train.drop_duplicates(keep="first",inplace=True)
train.shape[0]

48120

In [138]:
if(train.isnull().sum().sum()==0):
    print("No misssing value TRAIN")
else:
    train.fillna(method='ffill',inplace=True)
if(test.isnull().sum().sum()==0):
    print("No misssing value TEST")
else:
    test.fillna(method='ffill',inplace=True)

No misssing value TRAIN
No misssing value TEST


In [139]:
train['DateTime'] = pd.to_datetime(train['DateTime'])
test['DateTime'] = pd.to_datetime(test['DateTime'])
train.dtypes,test.dtypes

(DateTime    datetime64[ns]
 Junction             int64
 Vehicles             int64
 ID                   int64
 dtype: object,
 DateTime    datetime64[ns]
 Junction             int64
 ID                   int64
 dtype: object)

In [140]:
train["Year"]= train['DateTime'].dt.year  
train["Month"]= train['DateTime'].dt.month  
train["Date"]= train['DateTime'].dt.day  
train["Hour"]= train['DateTime'].dt.hour  
train["Day"]= train['DateTime'].dt.dayofweek

test["Year"]= test['DateTime'].dt.year  
test["Month"]= test['DateTime'].dt.month  
test["Date"]= test['DateTime'].dt.day  
test["Hour"]= test['DateTime'].dt.hour  
test["Day"]= test['DateTime'].dt.dayofweek

In [141]:
train.sample(5)

,DateTime,Junction,Vehicles,ID,Year,Month,Date,Hour,Day
13859,2017-05-31 11:00:00,1,100,20170531111,2017,5,31,11,2
25741,2017-02-07 13:00:00,2,20,20170207132,2017,2,7,13,1
5755,2016-06-27 19:00:00,1,60,20160627191,2016,6,27,19,0
30751,2016-01-05 07:00:00,3,4,20160105073,2016,1,5,7,1
20514,2016-07-04 18:00:00,2,18,20160704182,2016,7,4,18,0


In [142]:
test.sample(5)

,DateTime,Junction,ID,Year,Month,Date,Hour,Day
4751,2017-09-13 23:00:00,2,20170913232,2017,9,13,23,2
7620,2017-09-10 12:00:00,3,20170910123,2017,9,10,12,6
7938,2017-09-23 18:00:00,3,20170923183,2017,9,23,18,5
7588,2017-09-09 04:00:00,3,20170909043,2017,9,9,4,5
9365,2017-07-22 05:00:00,4,20170722054,2017,7,22,5,5


In [143]:
train = train.drop(columns=['DateTime','ID'])
test = test.drop(columns=['DateTime','ID'])

In [144]:
train.head()

,Junction,Vehicles,Year,Month,Date,Hour,Day
0,1,15,2015,11,1,0,6
1,1,13,2015,11,1,1,6
2,1,10,2015,11,1,2,6
3,1,7,2015,11,1,3,6
4,1,9,2015,11,1,4,6


In [145]:
test.head()

,Junction,Year,Month,Date,Hour,Day
0,1,2017,7,1,0,5
1,1,2017,7,1,1,5
2,1,2017,7,1,2,5
3,1,2017,7,1,3,5
4,1,2017,7,1,4,5


In [152]:
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

### RANDOM FOREST

In [153]:
X = train.drop(columns=['Vehicles'])
y = train['Vehicles']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Train size: x:{len(X_train)} and y:{len(y_train)}")

Train size: x:38496 and y:38496


In [154]:
random_forest_model = RandomForestRegressor(n_estimators=50,max_depth=10 ,random_state=42)
random_forest_model.fit(X_train,y_train)
pred = random_forest_model.predict(X_val)
mse = mean_squared_error(y_val, pred)
mae = mean_absolute_error(y_val, pred)
mse,mae

(26.025951621794942, 3.0953516129844845)

In [149]:
r2 = r2_score(y_val, pred)
r2

0.9361399593817984

### LGBM

In [155]:
lgbm_model = LGBMRegressor(random_state=42)
lgbm_model.fit(X_train,y_train)
pred = lgbm_model.predict(X_val)
mse = mean_squared_error(y_val,pred)
r2 = r2_score(y_val,pred)
mae = mean_absolute_error(y_val, pred)
mse,r2,mae

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000360 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 85
[LightGBM] [Info] Number of data points in the train set: 38496, number of used features: 6
[LightGBM] [Info] Start training from score 22.883598


(22.02870312914759, 0.9459480330619069, 2.8522394796987185)

### XGBoost

In [157]:
xgb_model = XGBRegressor(learning_rate=0.1,max_depth=6, n_estimatiors=100, random_state=42)
xgb_model.fit(X_train,y_train)
pred = xgb_model.predict(X_val)
mse = mean_squared_error(y_val,pred)
r2 = r2_score(y_val,pred)
mae = mean_absolute_error(y_val, pred)
mse,r2,mae

C:\Users\Admin\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:08:37] WARNING: D:\bld\xgboost-split_1733179550881\work\src\learner.cc:740: 
Parameters: { "n_estimatiors" } are not used.

  warnings.warn(smsg, UserWarning)


(24.072373584765295, 0.9409334659576416, 2.893921015915753)